In [29]:
#get file paths from user
p1 =readline(STDIN)
p2= readline(STDIN)

function getMetrics(path)
    #open file and read contents into array of words
    f=open(path)
    stuff = readstring(f)
    words = split(stuff)

    #counters for words, sentences
    numWords=0
    numSentences=0
    wordLens=0 #keep track of characters
    wordcounts=Dict() #keep track of counts of each words

    stopwords = ("ourselves", "hers", "between", "yourself", "but", "again", "there", "about", "once", "during", "out", "very", "having", "with", "they", "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "most", "itself", "other", "off", "is", "s", "am", "or", "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we", "these", "your", "his", "through", "don", "nor", "me", "were", "her", "more", "himself", "this", "down", "should", "our", "their", "while", "above", "both", "up", "to", "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them", "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then", "that", "because", "what", "over", "why", "so", "can", "did", "not", "now", "under", "he", "you", "herself", "has", "just", "where", "too", "only", "myself", "which", "those", "i", "after", "few", "whom", "t", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it", "how", "further", "was", "here", "than")

    #loop through each word in the book
    for word in words
        numWords=numWords+1
        #if word ends in .!? its the end of a sentence
        if(word[endof(word)]=='!' || word[endof(word)]=='.' || word[endof(word)]=='?')
            numSentences=numSentences+1
        end
        #strip punctuation off of word and add it to dictionary
        sword=lowercase(strip(word, ['.','!','?','‘','\"','“',',']))
        wordLens+=length(sword)
        if(sword in stopwords)
            continue
        end
        if(haskey(wordcounts, sword))
            wordcounts[sword]=wordcounts[sword]+1
        else
            wordcounts[sword]=1
        end
    end

    #sort the counts by the values (high to low)
    orderedcounts=sort(collect(wordcounts), by = tuple -> last(tuple), rev=true)
    top=Dict()
    for i in 1:10
        top[orderedcounts[i][1]]=orderedcounts[i][2]
    end

    close(f)
    
    #filename, words, sentences, sentence length, word length, top 10 words
    return (path, numWords, numSentences, numWords/numSentences, wordLens/numWords, top)

end

metrics1=getMetrics(p1)
metrics2=getMetrics(p2)


STDIN> alice_in_wonderland.txt
STDIN> pride_and_prejudice.txt


("pride_and_prejudice.txt", 121567, 5774, 21.05420852095601, 4.478698989034854, Dict{Any,Any}(Pair{Any,Any}("mrs", 343),Pair{Any,Any}("miss", 283),Pair{Any,Any}("much", 315),Pair{Any,Any}("elizabeth", 565),Pair{Any,Any}("could", 521),Pair{Any,Any}("darcy", 340),Pair{Any,Any}("must", 304),Pair{Any,Any}("said", 377),Pair{Any,Any}("would", 465),Pair{Any,Any}("mr", 781)…))

In [30]:
function similarity(metrics1, metrics2)
    #NUMWORDS--out of 2 possible points
    n1=metrics1[2]
    n2=metrics2[2]
    if(n1>n2)
        ratio=n1/n2
    else
        ratio=n2/n1
    end
    if(ratio>4)
        wordScore=0
    else
        wordScore=2/ratio
    end

    #SENTENCE LENGTH--out of 3 possible points
    n1=metrics1[4]
    n2=metrics2[4]
    if(n1>n2)
        ratio=n1/n2
    else
        ratio=n2/n1
    end
    if(ratio>2)
        sentenceScore=0
    else
        sentenceScore=(3/ratio)-0.33
    end

    #WORD LENGTH--out of 4 possible points
    n1=metrics1[5]
    n2=metrics2[5]
    dif=abs(n1-n2)
    if(dif>2)
        lenScore=0
    elseif(dif<.1)
        lenScore=4
    else
        lenScore=4/(10*dif)
    end

    #TOP WORDS--out of 1 possible point
    w1=keys(metrics1[6])
    w2=keys(metrics2[6])
    overlapScore=0
    for word in w1
        if(word in w2)
            overlapScore=overlapScore+1
        end
    end
    overlapScore=overlapScore/10

    total=overlapScore+lenScore+sentenceScore+wordScore
    return total
end

score = similarity(metrics1, metrics2)

5.0622284918877725

In [31]:
function printResults(metrics)
    filename = metrics[1]
    numWords = metrics[2]
    numSentences = metrics[3]
    lenSentences = metrics[4]
    topTen = sort(collect(metrics[6]), by = tuple -> last(tuple), rev=true)
    
    println("--------------------------------------------------------")
    println("            FILENAME: $filename")
    println("         TOTAL WORDS: $numWords")
    println("     TOTAL SENTENCES: $numSentences")
    println("AVG. SENTENCE LENGTH: $lenSentences")
    println("        TOP 10 WORDS: $(topTen[1][1]) $(topTen[1][2])")
    println("                      $(topTen[2][1]) $(topTen[2][2])")
    println("                      $(topTen[3][1]) $(topTen[3][2])")
    println("                      $(topTen[4][1]) $(topTen[4][2])")
    println("                      $(topTen[5][1]) $(topTen[5][2])")
    println("                      $(topTen[6][1]) $(topTen[6][2])")
    println("                      $(topTen[7][1]) $(topTen[7][2])")
    println("                      $(topTen[8][1]) $(topTen[8][2])")
    println("                      $(topTen[9][1]) $(topTen[9][2])")
    println("                      $(topTen[10][1]) $(topTen[10][2])")
    
end

printResults (generic function with 1 method)

In [32]:
printResults(metrics1)
printResults(metrics2)
println("--------------------------------------------------------")
println("          SIMILARITY: $score")

--------------------------------------------------------
            FILENAME: alice_in_wonderland.txt
         TOTAL WORDS: 29465
     TOTAL SENTENCES: 1083
AVG. SENTENCE LENGTH: 27.206832871652818
        TOP 10 WORDS: said 451
                      alice 354
                      little 125
                      one 94
                      project 86
                      went 83
                      would 82
                      could 78
                      like 77
                      thought 71
--------------------------------------------------------
            FILENAME: pride_and_prejudice.txt
         TOTAL WORDS: 121567
     TOTAL SENTENCES: 5774
AVG. SENTENCE LENGTH: 21.05420852095601
        TOP 10 WORDS: mr 781
                      elizabeth 565
                      could 521
                      would 465
                      said 377
                      mrs 343
                      darcy 340
                      much 315
                      must 304
     